In [1]:
pip install torch scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(0)
torch.manual_seed(0)

n_samples = 1000
X = np.random.rand(n_samples, 1) * 10
y = np.sin(X).ravel() + np.random.randn(n_samples) * 0.1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
np.random.seed(0)
torch.manual_seed(0)

n_samples = 1000
X = np.random.rand(n_samples, 1) * 10
y = np.sin(X).ravel() + np.random.randn(n_samples) * 0.1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
def quantile_loss(pred, target, quantiles):
    assert not target.requires_grad
    assert pred.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - pred[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
class QuantileRegressionModel(nn.Module):
    def __init__(self):
        super(QuantileRegressionModel, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)  # Output two quantiles: q_low, q_high
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = QuantileRegressionModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
quantiles = [0.025, 0.975]

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = quantile_loss(output, y_train, quantiles)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.1551
Epoch [20/100], Loss: 0.1091
Epoch [30/100], Loss: 0.0906
Epoch [40/100], Loss: 0.0875
Epoch [50/100], Loss: 0.0812
Epoch [60/100], Loss: 0.0758
Epoch [70/100], Loss: 0.0713
Epoch [80/100], Loss: 0.0669
Epoch [90/100], Loss: 0.0626
Epoch [100/100], Loss: 0.0582


In [8]:
with torch.no_grad():
    model.eval()
    preds = model(X_test).numpy()
    lower_preds = preds[:, 0]
    upper_preds = preds[:, 1]
    
    # Calculate nonconformity scores (absolute errors)
    calibration_errors = np.maximum(lower_preds - y_test.numpy(), y_test.numpy() - upper_preds)
    print(calibration_errors)
    # Compute the q_hat quantile of the calibration errors
    alpha = 0.06  # Coverage level
    q_hat = np.quantile(calibration_errors, 1 - alpha)
    print(q_hat)
    # Generate conformal intervals
    lower_bound = lower_preds - q_hat
    upper_bound = upper_preds + q_hat
    coverage=0
    print(f"Prediction intervals with {1-alpha} coverage:")
    for i in range(len(lower_bound)):  # Show a few examples
        #print(f"Predicted interval: [{lower_bound[i]:.3f}, {upper_bound[i]:.3f}], True value: {y_test[i]:.3f}")
        if y_test[i]>=lower_bound[i] and y_test[i]<=upper_bound[i]:
           coverage=coverage+1
    print("coverage is {:.6f}".format(coverage/len(lower_bound)))
    print(len(lower_bound))

[-0.69140375 -0.25012094 -0.41708943 -0.09124333 -0.07694805 -0.06675088
 -0.24454927 -0.6003647  -0.3057853  -0.7970952  -1.0392644  -0.5413891
 -0.22948724 -0.18526882 -0.447169   -0.13634866 -0.6917387  -0.13946998
 -0.17674935 -0.4445994  -0.5825853  -0.23574722 -0.57265997 -1.2963347
 -1.6399329  -0.14971668 -0.49162912 -0.625566   -0.10749876 -0.5033419
 -0.34127557 -0.23014218 -0.39188123 -0.22816372 -0.49552715 -0.4630437
 -0.9793608  -0.6303951  -0.3391699  -0.28044105 -0.43031162 -0.8957916
 -0.3836431  -1.5201294  -1.5882665  -0.17266965 -0.40139192 -0.60008913
  0.03701425 -0.34792763 -1.3249104  -0.15087557 -0.27941048 -0.09182686
 -0.26042867 -0.4675847  -0.6814906  -0.27468395 -0.6362156  -0.07213223
 -0.16245896 -0.09952539 -1.1040926  -0.9651578  -1.2106256  -0.25163978
 -0.20393777  0.02909541 -1.6940236  -0.8861201  -0.6505639  -0.06454617
 -0.35557985 -1.713304   -0.3953836  -0.98340905 -0.572245   -0.6039212
 -1.0666592  -0.3247531  -0.18688112 -0.35708988 -1.46997

In [1]:
import torch

# 假设 y_true 和 y_pred 已经定义
# y_true: [200, 1, 5, 1], y_pred: [200, 10, 5, 1]
# 示例创建随机输入
y_true = torch.rand(200, 1, 5, 1)
y_pred = torch.rand(200, 10, 5, 1)

# 计算 L1 距离
# 广播计算: [200, 1, 5, 1] 和 [200, 10, 5, 1] -> [200, 10, 5, 1]
distances = torch.norm(y_true.squeeze(-1) - y_pred.squeeze(-1), dim=-1)  # ||y_pred - y_true||_2  # 计算逐元素绝对值差
print(distances.shape)
#l1_distances_sum = l1_distances.sum(dim=2)  # 按维度 2 (特征维度) 求和，结果为 [200, 10, 1]

# 找到最小 L1 距离
#min_l1_distances, _ = l1_distances_sum.min(dim=1)  # 对维度 1 (10 个估计) 求最小值，结果为 [200, 1]

# 展示结果
#print(min_l1_distances.shape)  # [200, 1]

ModuleNotFoundError: No module named 'torch'

In [2]:
pip install torch

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 